# MPlots
Jugando con los MatrixProfile & MPlots comenzando con el tutorial de Stumpy
https://stumpy.readthedocs.io/en/latest/Tutorial_The_Matrix_Profile.html

In [1]:
import numpy as np
import dvats.memory as mem
import dvats.mplots as mp

KeyboardInterrupt: 

In [ ]:
mps = mp.MatrixProfiles()

In [ ]:
#Common
mps.data = np.random.rand(10000)
mps.subsequence_len = 50
print("Ts ~ ", mps.data.shape)
print("Expected mp length: ", int(np.floor(mps.data.shape[0]-mps.subsequence_len+1)))

In [ ]:
# Compute & Save Matrix Profile using both stumpy.stump & scamp
mps.compute('stump',  True, True, True)
print("Total MPs: ", len(mps.matrix_profiles))
print(mps.matrix_profiles[0])
mps.compute('scamp',  True, True, True)
print("Total MPs: ", len(mps.matrix_profiles))
print(mps.matrix_profiles[1])

In [ ]:
mps.plot([])

### Ejemplo de STUMPY Basics - Analyzing Motifs and Anomalies with STUMP and SCAMP
https://stumpy.readthedocs.io/en/latest/Tutorial_The_Matrix_Profile.html

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt

plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

In [ ]:
steam_df = pd.read_csv("https://zenodo.org/record/4273921/files/STUMPY_Basics_steamgen.csv?download=1")
steam_df.head()

In [ ]:
mp.plot_dataFrame('Steamgen Dataset', steam_df, [3])

In [ ]:
mp.plot_dataFrame_compareSubsequences('Steamgen Dataset', steam_df, 3, 640, 643, 8724)

In [ ]:
steam_mps = mp.MatrixProfiles()
steam_mps.subsequence_len = 640
steam_mps.data = steam_df['steam flow']
steam_mps.compute('stump', print_flag = False, debug = False, timed = True)
steam_mps.compute('scamp', print_flag = False, debug = False, timed = True)

In [ ]:
steam_mps.plot()

In [ ]:
print("Ts ~ ", steam_mps.data.shape)
print("Expected mp length: ", int(np.floor(steam_mps.data.shape[0]-steam_mps.subsequence_len+1)))
print("mp_stumpy ~", steam_mps.matrix_profiles[0].matrix_profile.shape)
print("mp_scamp ~", steam_mps.matrix_profiles[1].matrix_profile.shape)
print(steam_mps.matrix_profiles[0])
print(steam_mps.matrix_profiles[1])

In [ ]:
stumpy_idxs = steam_mps.get_motif_idx(0)
scamp_idxs = steam_mps.get_motif_idx(1)

In [ ]:
print(f"The motif (according to stumpy) is located at index {stumpy_idxs[0]}")
print(f"The motif (according to scamp) is located at index {scamp_idxs[0]}")

### --> Tiene sentido que haya salido diferente por tema de decimales... 
> ¿Serán vecinos?

In [ ]:
print(f"The nearest neighbor (stumpy) is located at index {stumpy_idxs[1]}")
print(f"The nearest neighbor (scamp) is located at index {scamp_idxs[1]}")

¡Son los vecinos más cercanos en ambos casos!
De hecho, si imprimimos los valores, la z-distancia euclídea es la misma hasta donde podemos ver...

In [ ]:
print( steam_mps.matrix_profiles[1].matrix_profile[stumpy_idxs[0]])
print( steam_mps.matrix_profiles[1].matrix_profile[stumpy_idxs[1]])
print( steam_mps.matrix_profiles[0].matrix_profile[scamp_idxs[0]])
print( steam_mps.matrix_profiles[0].matrix_profile[scamp_idxs[1]])

In [ ]:
steam_mps.plot_motif('Steam flow', 0, stumpy_idxs[0], stumpy_idxs[1], '30', '10')
steam_mps.plot_motif('Steam flow', 1, stumpy_idxs[0], stumpy_idxs[1], '30', '10')

## Discord / Anomaly

In [ ]:
#mp_stumpy_sorted = np.argsort( steam_mps.matrix_profiles[0].matrix_profile )
#mp_scamp_sorted =  np.argsort( steam_mps.matrix_profiles[1].matrix_profile )

#discord_idx_stumpy = mp_stumpy_sorted[-1]

#discord_idx_scamp = mp_scamp_sorted[-1]
#print( steam_mps.matrix_profiles[1].matrix_profile[stumpy_idxs[0]])
steam_mps.get_anomaly_idx(0)
steam_mps.get_anomaly_idx(1)
discord_idx_stumpy = steam_mps.matrix_profiles[0].discord_idx
discord_idx_scamp = steam_mps.matrix_profiles[1].discord_idx
print(f"Stumpy: The discord is located at index {discord_idx_stumpy}")
print(f"Scamp: The discord is located at index {discord_idx_scamp}")

In [ ]:
print("Stumpy | Discord: [", steam_mps.matrix_profiles[0].matrix_profile[discord_idx_stumpy],",", steam_mps.matrix_profiles[0].index[discord_idx_stumpy], "]")
print("Scamp |  Discord: [", steam_mps.matrix_profiles[1].matrix_profile[discord_idx_scamp], ",",steam_mps.matrix_profiles[1].index[discord_idx_scamp], "]")

The subsequence located at this global maximum is also referred to as a discord, novelty, or “potential anomaly”:

In [ ]:
steam_mps.plot_motif(
    'Steam Flow', 
    0, 
    steam_mps.matrix_profiles[0].discord_idx, 
    steam_mps.matrix_profiles[0].discord_nearest_neighbor_idx, 
    '20',
    '15'
)

In [ ]:
steam_mps.plot_motif(
    'Steam Flow', 
    1, 
    steam_mps.matrix_profiles[1].discord_idx, 
    steam_mps.matrix_profiles[1].discord_nearest_neighbor_idx, 
    '20',
    '15'
)

In [ ]:
steam_mps.plot_motif(
    'Steam Flow', 
    0, 
    steam_mps.matrix_profiles[0].discord_idx, 
    steam_mps.matrix_profiles[0].discord_nearest_neighbor_idx_left, 
    '20',
    '15'
)

Intentando ver la matriz Distance Profile al completo

In [ ]:
steam_flow = steam_df['steam flow'].values
#No nans
steam_flow = [ 0 if np.isnan(x) else x for x in steam_flow ]

## Cogiendo Distance Matrix al completo 
> En busca del MPlot

In [ ]:
steam_mps.data
steam_mps.subsequence_len
# Ensure no nans 
steam_mps.data = [ 0 if np.isnan(x) else x for x in steam_mps.data ]

In [ ]:
#n = len(steam_flow) 
#subsequence_len = m
#print("Expected: ", len(steam_flow) - m + 1)

In [ ]:
#MPlot_matrix_stumpy = np.empty((n - m + 1, n - m + 1))
#MPlot_matrix_scamp = np.empty((n - m + 1, n - m + 1))

In [ ]:
mplot_stumpy = mp.MatrixProfilePlot(data = steam_mps.data, subsequence_len = steam_mps.subsequence_len)
mplot_scamp  = mp.MatrixProfilePlot(data = steam_mps.data, subsequence_len = steam_mps.subsequence_len)

In [ ]:
reference_idx = np.random.randint(low=0, high=len(steam_flow) - steam_mps.subsequence_len)
reference_subseq = steam_flow[reference_idx:reference_idx + steam_mps.subsequence_len]

### Calcular el Distance Profile para cada subsecuencia en la serie temporal respecto a la secuencia de referencia

In [ ]:
mplot_stumpy.compute_similarity_matrix(
    subsequence_len = mplot_stumpy.subsequence_len,
    reference_seq   = reference_subseq,
    method          = 'stump',
    timed           = True,
    print_flag      = True
)

In [ ]:
mplot_scamp.compute_similarity_matrix(
    subsequence_len = mplot_stumpy.subsequence_len,
    reference_seq   = reference_subseq,
    method          = 'scamp',
    timed           = True,
    print_flag      = True
)

In [ ]:
mplot_stumpy.plot('Steam flow', "Stumpy")
mplot_scamp.plot('Steam flow', "Scamp")

### Calcular el Distance Profile para cada subsecuencia en la serie temporal al completo

In [ ]:
mplot_stumpy.compute_similarity_matrix(
    subsequence_len = mplot_stumpy.subsequence_len,
    #reference_seq   = reference_subseq,
    reference_seq   = None,
    method          = 'stump',
    timed           = True,
    print_flag      = True
)

In [ ]:
mplot_stumpy.plot('Steam flow', "Stumpy")

In [ ]:
mplot_scamp.compute_similarity_matrix(
    subsequence_len = mplot_stumpy.subsequence_len,
    reference_seq   = None, #reference_subseq,
    method          = 'scamp',
    timed           = True,
    print_flag      = True
)

In [ ]:
print(len(steam_flow))
print(mplot_stumpy.similarity_matrix.shape)
print(mplot_scamp.similarity_matrix.shape)

In [ ]:
mplot_scamp.plot('Steam flow', "Scamp")